# Решение №152989242

Инструкции по запуску:
1) Установить зависимости
2) Запустить остальные ячейки в последовательном порядке
3) Вывод ответа в файле `output.txt`

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
import datetime


In [ ]:
df = pd.read_csv('train.csv', parse_dates=True)

df['total'][0] = 154140 - 3700
for i in range(0, len(df['total']) - 1):
    if df['total'][i] == "?":
        cur, next, prev = df['total'][i], i + 1, i - 1
        while df['total'][next] == "?":
            next += 1
        while df['total'][prev] == "?":
            prev -= 1

        generated = np.linspace(
            int(df['total'][prev]),
            int(df['total'][next]),
            next - prev,
            endpoint=False)
        for i in range(prev + 1, next):
            df['total'][i] = generated[i - prev]

df['total'] = df['total'].astype(float)
df['datetime'] = pd.to_datetime(df['datetime'])


In [ ]:
inp = []
ans = []
for i in range(24*365+24, len(df['datetime'][24*365+24:])):
    tmp = [df['datetime'][i].hour,
           df['datetime'][i].day,
           df['datetime'][i].dayofweek,
           df['datetime'][i].month,
           df['datetime'][i].dayofyear,
           df['datetime'][i].weekofyear,
           df['datetime'][i].quarter,
           df['datetime'][i].year]
    for j in range(1, 25):
        tmp.append(df['total'][i-24*365-j])
    for j in range(1, 25):
        tmp.append(df['total'][i-j])
    for j in range(1, 25):
        tmp.append(df['total'][i-24*30-j])
    for j in range(1, 25):
        tmp.append(df['total'][i-24*7-j])
    inp.append(tmp)
    ans.append(df['total'][i])


In [ ]:
model = CatBoostRegressor(iterations=10000, random_seed=700)
model.fit(inp, ans)


In [ ]:
total = list(df['total'])
pred = []
start = datetime.datetime(2009, 1, 1)
for i in range(4344):
    cur_t = pd.Timestamp(start + datetime.timedelta(hours=i))
    get = [cur_t.hour,
           cur_t.day,
           cur_t.dayofweek,
           cur_t.month,
           cur_t.dayofyear,
           cur_t.weekofyear,
           cur_t.quarter,
           cur_t.year]
    for j in range(1, 25):
        get.append(total[i-24*365-j])
    for j in range(1, 25):
        get.append(total[i-j])
    for j in range(1, 25):
        get.append(total[i-24*30-j])
    for j in range(1, 25):
        get.append(total[i-24*7-j])
    a = model.predict([get, ])[0]
    total.append(a)
    pred.append(a)


In [ ]:
with open('output.txt', 'w') as f:
    for i in pred:
        f.write(str(i)+'\n')
